In [66]:
from dotenv import load_dotenv
from google import genai
from groq import Groq
from openai import OpenAI
from pathlib import Path
import anthropic
import os

load_dotenv()

True

In [ ]:
# connect to providers
client_anthropic = anthropic.Anthropic(api_key=os.environ.get('ANTHROPIC_KEY'))
client_google = genai.Client(api_key=os.environ.get('GOOGLE_KEY'))
client_groq = Groq(api_key=os.environ.get('GROQ_KEY'))
client_openai = OpenAI(api_key=os.environ.get('OPENAI_KEY'))

# get the lists of models
models_anthropic = client_anthropic.models.list(limit=20)
models_google = client_google.models.list()
models_groq = client_groq.models.list()
models_openai = client_openai.models.list()

# extract model ids
anthropic_model_id = [model.id for model in models_anthropic.data]
google_model_id = [model.name for model in models_google]
groq_model_id = [model.id for model in models_groq.data]
openai_model_id = [model.id for model in models_openai.data]

In [68]:
model_ids = {
    'ANTHROPIC': anthropic_model_id,
    'GOOGLE': google_model_id,
    'GROQ': groq_model_id,
    'OPENAI': openai_model_id,
}

def normalize_name(name: str, seen: set) -> str:
    norm = name.replace('-', '_').replace('.', '_').replace('/', '_').upper()
    if norm in seen:
        raise ValueError(f'Duplicate name: {norm} from {name}')
    seen.add(norm)
    return norm

provider_enum = [
    'from enum import Enum, auto\n\n',
    'class Provider(Enum):\n'
]

for provider in sorted(model_ids.keys()):
    provider_enum.append(f'    {provider} = auto()\n')

model_enum = ['\nclass Model(Enum):\n']
model_to_provider = ['\nmodel_to_provider = {\n']

seen = set()
for provider, ids in model_ids.items():
    for model in sorted(ids):
        model_name = normalize_name(model, seen)
        model_enum.append(f'    {model_name} = \'{model}\'\n')
        model_to_provider.append(f'    Model.{model_name}: Provider.{provider},\n')

model_to_provider.append('}\n')

output_path = Path('enums.py')
output_path.write_text( ''.join(provider_enum + model_enum + model_to_provider) )

18289